<a href="https://colab.research.google.com/github/Dospalko/datascience/blob/main/LangChainaHugginFaceIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalacia balickov

In [2]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [5]:
import torch
import getpass
import os
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda:0'

In [6]:
torch.random.manual_seed(42)

Definicia tokenov

In [8]:
os.environ["HUGGINGFACE_TOKEN"] = getpass.getpass("Enter your token:")

Enter your token:··········


LOADING the MODEL

In [7]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

TOKENIZER

In [9]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [10]:
#Pipeline connection
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Parametre pre text generation

In [11]:
generation_args={
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1,
    "do_sample":True,
}

In [12]:
prompt = "What is quantum computing?"
output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


In [13]:
output

[{'generated_text': ' Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers, which use bits as the basic unit of information (represented as 0 or 1), quantum computers use quantum bits, or qubits, which can exist in multiple states simultaneously. This allows quantum computers to perform certain calculations much faster than classical computers. How does quantum computing differ from classical computing? Quantum computing differs from classical computing in several key ways. First, quantum computers use qubits, which can exist in multiple states simultaneously, while classical computers use bits that can only be in one of two states (0 or 1). This allows quantum computers to perform certain calculations much faster than classical computers. Second, quantum computers can perform parallel processing, which means they can process multiple calculations at the same ti

In [14]:
print(output[0]['generated_text'])

 Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers, which use bits as the basic unit of information (represented as 0 or 1), quantum computers use quantum bits, or qubits, which can exist in multiple states simultaneously. This allows quantum computers to perform certain calculations much faster than classical computers. How does quantum computing differ from classical computing? Quantum computing differs from classical computing in several key ways. First, quantum computers use qubits, which can exist in multiple states simultaneously, while classical computers use bits that can only be in one of two states (0 or 1). This allows quantum computers to perform certain calculations much faster than classical computers. Second, quantum computers can perform parallel processing, which means they can process multiple calculations at the same time. This is because q

# Templates and prompt enginneering

In [15]:
prompt

'What is quantum computing?'

In [16]:
template = """
<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [17]:
template

'\n<|system|>\nYou are a helpful assistant.<|end|>\n<|user|>\n"What is quantum computing?"<|end|>\n<|assistant|>'

In [18]:
output = pipe(template, **generation_args)
print(output[0]["generated_text"])

 Quantum computing is a type of computing that takes advantage of the quantum states of subatomic particles to store information. Unlike classical computers, which use bits as the smallest unit of data (represented by 0s or 1s), quantum computers use quantum bits, or qubits, which can exist in multiple states simultaneously due to the principle of superposition. This allows quantum computers to process a vast number of possibilities at once, potentially solving complex problems much faster than classical computers. Quantum computing also leverages other quantum phenomena such as entanglement and quantum interference to perform computations. The field is still in its early stages, but it has the potential to revolutionize areas such as cryptography, drug discovery, and optimization problems.


In [19]:
prompt = "What is AI" # @param {type:"string"}
template = """
<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)
output = pipe(template, **generation_args)
print(output[0]["generated_text"])

 AI, or artificial intelligence, refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind, such as learning and problem-solving. AI systems can range from simple, rule-based algorithms to complex neural networks that learn from data. These systems are used in various applications, including speech recognition, image analysis, autonomous vehicles, and personalized recommendations.


Message format

In [21]:
prompt = "What is AI?"
message = [{
    "role": "system",
    "content": "You are a helpful assistant."
}, {
    "role": "user",
    "content": prompt
}]
output = pipe(message, **generation_args)
print(output[0]["generated_text"])


 AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It involves the development of algorithms and models that enable computers to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be categorized into two main types: narrow or weak AI, which is designed to perform specific tasks, and general or strong AI, which aims to possess human-like intelligence across a wide range of domains.


# QUANTIZATION

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
401 Client Error. (Request ID: Root=1-674db91b-2060911d147897c66848fc34;172fa503-22fa-47c2-9268-e1c2ce041ed7)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [11]:
prompt = ("Who was the first person in spae?")
meseges = [{"role":"user", "content":prompt}]

In [12]:
encodeds = tokenizer.apply_chat_template(meseges, return_tensors="pt")
model_inputs = encodeds.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]
res

NameError: name 'tokenizer' is not defined